In [1]:
import urllib.request, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import numpy as np
import shutil
import os

In [3]:
soup_from_txt = BeautifulSoup(Path('DataET_puhas/2609kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd" >

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=EmulateIE10" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>CURIA - Documents
        </title>
<link class="component" href="http://curia.europa.eu/juris/a4j/s/3_3_3.Finalorg/richfaces/renderkit/html/css/basic_classes.xcss/DATB/eAFLUXraH7p8hjQAENMDqg__;jsessionid=C74CA4C63498F0160953C98289FC5F38" rel="stylesheet" type="text/css"/><link class="component" href="http://curia.europa.eu/juris/a4j/s/3_3_3.Finalorg/richfaces/renderkit/html/css/extended_classes.xcss/DATB/eAFLUXraH7p8hjQAENMDqg__;jsessionid=C74CA4C63498F0160953C98289FC5F38" media="rich-extended-skinning" rel="stylesheet" type="text/css"/><script id="org.ajax4jsf.queue_script" type="text/javascript">if (typeof A4J != 'undefined') { if (A4J.AJAX) { with (A4J.AJAX) {if (!EventQueue.getQueue('or

In [44]:
def sisu_tabeliks(supp):
    abilist=[]
    tag_nimi=''
    Pealirjad_list=''
    Tekst_list=''
    Pealkirjad = [['C06Titre3'], ['C27Niveau6titre2'], ['C34Niveau7titre3'], ['Ctoc2'], ['C04Titre1'], ['CTOC4'], ['Ctoc1'], 
                  ['C06Titre4'], ['CTOC6'], ['C24Titre4'], ['C23Titre3'], ['C21Titrenumerote1'], ['C21Titre1'], 
                  ['C28Niveau6titre3'], ['C22Titrenumerote2'], ['C21Niveau5titre1'], ['C07Titre5'],['CTOC1'], 
                  ['C25Niveau5titre5'], ['Ctoc3'], ['C26Titre6'], ['C25Titrenumerote5'], ['CTOC2'], ['CTOC5'], 
                  ['C35Niveau7titre4'], ['C22Niveau5titre2'], ['C33Niveau7titre2'], ['CTOC3'], ['C24Titrenumerote4'], 
                  ['C22Titre2'], ['C30Niveau6titre5'], ['C23Niveau5titre3'], ['C26Titrenumerote6'], ['C05Titre2'], 
                  ['C29Niveau6titre4'], ['CTOC7'], ['C31Niveau6titre6'], ['C27Titrenumerote7'], ['Ctoc6'], ['C37Niveau7titre6'],
                  ['Cheading2'], ['Cheading1'], ['Cheading3'], ['C23Titrenumerote3'], ['C25Titre5'], ['Ctoc7'], ['C07Titre4'], 
                  ['Ctoc5'], ['C24Niveau5titre4'], ['C38Niveau7titre7'], ['C36Niveau7titre5'], ['C05Titre3'], ['Ctoc4'], 
                  ['C32Niveau7titre1'], ['C26Niveau6titre1']]
    Tekst = [['15Marge2avectiretlong'], ['C74Alineagrandespacement'], ['C12Tiretlong3'], ['C03Tiretlong'], ['C02RPAlineaAltA'],
             ['Cnull'], ['C27Autre'], ['C39Autre'], ['C10Tiretlong1'], 
             ['C01PointAltN'], ['C50Alineaencadre'], ['C12Marge1avectiretlong'], ['C18Marge3avectiretlong'], ['C15Margeretrait0'],
             ['C20Centreitalique'], ['C38Marge1doubleretrait'], ['C80TableLibre'], ['C16Marge3'], ['C09Pointtypographiqueniv1'], 
             ['C28Pointnumerotemanuel'], ['C39autre'], ['C13Marge2'], ['C11Marge1avecretrait'], 
             ['C32Pointnumerotemanuel'], ['C11Tiretlong2'], ['C01PointnumeroteAltN'], ['C08Question'], ['C17Marge3avecretrait'], ['C09Marge0avecretrait'], ['C10Marge1'], ['C41Marge0doubleretrait'], 
             ['C29Marge0doubleretrait'], ['C14Marge2avecretrait'], ['C80Autre'],
             ['C40Pointnumerotemanuel']]
    for elem in supp.findAll('p'):
        if elem.has_attr('class'):
            #print(elem['class'])
            if elem['class'] in Pealkirjad:
                tag_nimi=elem['class'][0]
                Pealkirjad_list = elem.get_text().replace(u'\xa0', u' ')
                abilist.append([tag_nimi, Pealkirjad_list])
            elif elem['class'] in Tekst:
                Tekst_list = elem.get_text().replace(u'\xa0', u' ')
                tag_nimi=elem['class'][0]
                abilist.append([tag_nimi, Tekst_list])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag', 'Lõigu sisu']
    return df_temp
sisu_tabeliks(soup_from_txt)     

,Tag,Lõigu sisu
0,C04Titre1,Õiguslik raamistik
1,C01PointnumeroteAltN,1 EÜ artikkel 255 näeb ette:
2,C01PointnumeroteAltN,2 Euroopa Parlamendi ja nõukogu 30. mai...
3,C01PointnumeroteAltN,3 Määruse nr 1049/2001 artikkel 2 sätes...
4,C01PointnumeroteAltN,4 Määruse nr 1049/2001 artikkel 3 kõlab:
...,...,...
73,C04Titre1,Kohtukulud
74,C01PointnumeroteAltN,58 Kodukorra artikli 87 lõike 2 järgi on ...
75,C01PointnumeroteAltN,"59 Käesolevas asjas tuleb, nagu Esimese A..."
76,C01PointnumeroteAltN,"60 Esimese Astme Kohus arvestab siiski, e..."


In [45]:
def kohtuasja_nr(supp):
    kohtuasja_nr = supp.find('p', class_='C02AlineaAltA')
    kohtuasja_nr_1=kohtuasja_nr.get_text().replace(u'\xa0', u' ').replace(',','').replace('Kohtuasjas ', '').replace('Liidetud kohtuasjades ', '')
    return kohtuasja_nr_1

kohtuasja_nr(soup_from_txt)

'T‑76/02'

In [46]:
#resolutsioon
def resolutsioon(supp):
    tag=''
    abilist=[]
    resolutsioon1=''
    resolutsioon = supp.findAll('p', class_=["C30Dispositifalinea", "C08Dispositif", "C31Dispositiftiretlong","C32Dispositifmarge1","C36Dispositifmarge2avectiretlong","C39Dispositifmarge1doubleretrait", "C37Dispositifmarge0doubleretrait", "C42Dispositifalinea","C02AlineaAltACharChar","C45Dispositifmarge1avectiretlong", "C44Dispositifmarge1avecretrait","C43Dispositiftiretlong","C33Dispositifmarge1avecretrait", "C40Dispositifmarge0avecretrait","C48DispositifIntroduction","C34Dispositifmarge1avectiretlong","C41DispositifIntroduction"])
    for elem in resolutsioon:
        tag=elem['class'][0]
        resolutsioon1=elem.get_text().replace(u'\xa0', u' ')
        abilist.append([tag, resolutsioon1])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag','Lõigu sisu']
    df_temp['Pealkiri1'] = 'Resolutsioon' 
    return df_temp
    
resolutsioon(soup_from_txt)

,Tag,Lõigu sisu,Pealkiri1
0,C48DispositifIntroduction,Esitatud põhjendustest lähtudes,Resolutsioon
1,C08Dispositif,1. Jätta hagi rahuldamata.,Resolutsioon
2,C08Dispositif,2. Hageja kannab ise poole oma kohtukulud...,Resolutsioon


In [47]:
#pooled     
def poolte_nimed(supp):
    tag = 'C02AlineaAltA'
    abilist = []
    poolte_nimed = ''
    kogu_algus = supp.findAll('p', class_='C02AlineaAltA')
    poolte_nimed = []
    for elem in kogu_algus:
        if elem.findAll('b'):
            poolte_nimed=elem.get_text()
            abilist.append([tag, poolte_nimed])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag','Lõigu sisu']
    df_temp['Pealkiri1'] = 'Poolte_nimed' 
    return df_temp

poolte_nimed(soup_from_txt)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [48]:
#kuupäev
def kuupaev(supp):
    kuupaev = supp.findAll('p', class_="C19Centre")
    kuupaev1 = kuupaev[1].get_text().replace('(*)', '').replace(u'\xa0', u' ')
    df_temp=pd.DataFrame(columns=['Lõigu sisu','Pealkiri1'], index=range(1))
    df_temp['Lõigu sisu'] = kuupaev1
    df_temp['Pealkiri1'] = 'Kuupaev'
    df_temp['Tag'] = 'C19Centre'
    return df_temp

kuupaev(soup_from_txt)

,Lõigu sisu,Pealkiri1,Tag
0,17. september 2003,Kuupaev,C19Centre


In [49]:
#Viited märksõnadele
def viited_marksonale(supp):
    viited = supp.findAll('p', class_='C71Indicateur')
    #print(viited)
    viited1 = viited[0].get_text().replace(u'\xa0', u' ')
    df_temp=pd.DataFrame(columns=['Lõigu sisu','Pealkiri1'], index=range(1))
    df_temp['Lõigu sisu'] = viited1
    df_temp['Pealkiri1'] = 'Viited'
    df_temp['Tag'] = 'C71Indicateur'
    return df_temp

viited_marksonale(soup_from_txt)

,Lõigu sisu,Pealkiri1,Tag
0,Määrus (EÜ) nr 1049/2001 – Juurdepääs dokument...,Viited,C71Indicateur


In [50]:
#koda
def koda(supp):
    koda = supp.findAll('p', class_=["C19Centre", "C93Titredefinitif"])
    koda1 = koda[0].get_text().replace(',','')
    df_temp=pd.DataFrame(columns=['Lõigu sisu','Pealkiri1'], index=range(1))
    df_temp['Lõigu sisu'] = koda1
    df_temp['Pealkiri1'] = 'Koda'
    df_temp['Tag'] = koda[0]['class'][0]
    return df_temp
koda(soup_from_txt)

,Lõigu sisu,Pealkiri1,Tag
0,ESIMESE ASTME KOHTU OTSUS (neljas koda laienda...,Koda,C19Centre


In [51]:
#kohtumenetluse keel 
def kohtumenetluse_keel(supp):
    kohtumenetluse_keel = supp.findAll('p', class_=["C42FootnoteLangue", 'C49FootnoteLangue'])
    #print(kohtumenetluse_keel)
    keel = kohtumenetluse_keel[0].get_text().replace(u'\xa0', u' ').replace('* Kohtumenetluse keel: ', '').replace('*      Kohtumenetluse keel: ', '').replace('.', '')
    df_temp=pd.DataFrame(columns=['Lõigu sisu','Pealkiri1'], index=range(1))
    df_temp['Lõigu sisu'] = keel
    df_temp['Pealkiri1'] = 'Keel'
    df_temp['Tag'] = kohtumenetluse_keel[0]['class'][0]
    return df_temp
kohtumenetluse_keel(soup_from_txt)

,Lõigu sisu,Pealkiri1,Tag
0,itaalia,Keel,C49FootnoteLangue


In [20]:
def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    return df_temp#.iloc[1]['Lõigu sisu']
kohtulahend(soup_from_txt)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [102]:
pohitabel = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
for i in range(0):#23803):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    try:
        if os.path.isfile(Path('DataET_puhas/'+str(i)+'kohtulahend.txt')): #kontrolli kas fail eksisteerib
            #print('DataET_puhas/'+str(i)+'kohtulahend.txt')
            text=BeautifulSoup(Path('DataET_puhas/'+str(i)+'kohtulahend.txt').read_text(encoding='utf-8'))
            df_temp = df_temp.append(kohtulahend(text))
            df_temp['Kohtuasja_ID']=str(i)
            #print(df_temp)
            #print(pohitabel)
            i+=1
        else:
            i+=1
    except Exception as e:
        print('DataET_puhas/'+str(i)+'kohtulahend.txt')
        print(e)
    pohitabel=pohitabel.append(df_temp)
#df_temp.head(20)
pohitabel.head()

DataET_puhas/2148kohtulahend.txt
list index out of range
DataET_puhas/2383kohtulahend.txt
list index out of range
DataET_puhas/2609kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/2677kohtulahend.txt
list index out of range
DataET_puhas/4552kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/5024kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/6008kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/6106kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/6428kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/6485kohtulahend.txt
Length mismatch: Expected axis has 0 elements, new values have 2 elements
DataET_puhas/6629kohtulahend.txt
Length mismatch: Expected axis has 0 elements, 

,Kohtuasja_ID,Kohtuasja_nr,Lõigu sisu,Pealkiri1,Tag
0,0,C‑416/20 PPU,EUROOPA KOHTU OTSUS (neljas koda),Koda,C19Centre
1,0,C‑416/20 PPU,17. detsember 2020,Kuupaev,C19Centre
2,0,C‑416/20 PPU,Eelotsusetaotlus – Eelotsuse kiirmenetlus – Po...,Viited,C71Indicateur
3,0,C‑416/20 PPU,TR,Poolte_nimed,C02AlineaAltA
4,0,C‑416/20 PPU,"Generalstaatsanwaltschaft Hamburg,",Poolte_nimed,C02AlineaAltA


In [104]:
#pohitabel.to_csv('Pohitabel.csv', index=False)
#pohitabel.tail(30)

,Kohtuasja_ID,Kohtuasja_nr,Lõigu sisu,Pealkiri1,Tag
101,23801,F‑1/05,"72 Pealegi soovisid parlament, nõukogu ja...",NaN,C01PointnumeroteAltN
102,23801,F‑1/05,"73 Siinkohal tuleks märkida, et ükski üle...",NaN,C01PointnumeroteAltN
103,23801,F‑1/05,74 Selles osas piisava kaitse tagamiseks ...,NaN,C01PointnumeroteAltN
104,23801,F‑1/05,"75 Tuleks veel lisada, et pädeva asutuse ...",NaN,C01PointnumeroteAltN
105,23801,F‑1/05,"76 Peale selle tuleb tõdeda, et eelnevad ...",NaN,C01PointnumeroteAltN
106,23801,F‑1/05,77 Nendel tingimustel tuleb käesoleval ju...,NaN,C01PointnumeroteAltN
107,23801,F‑1/05,78 Ühelt poolt tuleneb vormilise põhjenda...,NaN,C01PointnumeroteAltN
108,23801,F‑1/05,79 Tähtajatu töölepinguga tööle võetud tö...,NaN,C01PointnumeroteAltN
109,23801,F‑1/05,80 Käesoleval juhul nähtub kohtutoimikust...,NaN,C01PointnumeroteAltN
110,23801,F‑1/05,81 Sellest tulenevalt tuleb põhjendamisko...,NaN,C01PointnumeroteAltN


In [21]:
#täiendavate andmete lisamine (neis kus keeled puudusid- 2148, 2677, 7153, 7740, 20405) 
#NBÜks haaval sisse loetud muutes numbrit!
soup_from_txt = BeautifulSoup(Path('DataET_puhas/20405kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt
def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    #df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    df_temp['Kohtuasja_ID']='20405'
    return df_temp#.iloc[1]['Lõigu sisu']
tabel=kohtulahend(soup_from_txt)
tabel
#tabel.to_csv('Proov.csv', index=False)
#tabel.to_csv('Pohitabel.csv', mode='a', header=False, index=False)

In [113]:
soup_from_txt = BeautifulSoup(Path('DataET_puhas/20429kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd" >

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=EmulateIE10" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>CURIA - Documents
        </title>
<link class="component" href="http://curia.europa.eu/juris/a4j/s/3_3_3.Finalorg/richfaces/renderkit/html/css/basic_classes.xcss/DATB/eAFLUXraH7p8hjQAENMDqg__;jsessionid=1C344BCEF4894AB91CFB2611A929A6DD" rel="stylesheet" type="text/css"/><link class="component" href="http://curia.europa.eu/juris/a4j/s/3_3_3.Finalorg/richfaces/renderkit/html/css/extended_classes.xcss/DATB/eAFLUXraH7p8hjQAENMDqg__;jsessionid=1C344BCEF4894AB91CFB2611A929A6DD" media="rich-extended-skinning" rel="stylesheet" type="text/css"/><script id="org.ajax4jsf.queue_script" type="text/javascript">if (typeof A4J != 'undefined') { if (A4J.AJAX) { with (A4J.AJAX) {if (!EventQueue.getQueue('or

In [54]:
#täiendavate andmete lisamine (6629, 6667, 6949, 7061, 7062, 7063, 7068, 7167, 7170, 17935, 17950, 20429-b tagi pole poolte_nimede jaoks)
#Käsitsi oleks vaja muuta ära, millisest vahemikust ta poolte nimed saab!
#Sisse loetud!
soup_from_txt = BeautifulSoup(Path('DataET_puhas/20429kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')

#pooled     
def poolte_nimed(supp):
    tag = 'C02AlineaAltA'
    abilist = []
    poolte_nimed = ''
    kogu_algus = supp.findAll('p', class_='C02AlineaAltA')
    poolte_nimed = []
    for elem in kogu_algus:
        poolte_nimed=elem.get_text()
        abilist.append([tag, poolte_nimed])
    df_temp=pd.DataFrame(abilist[1:3])
    #df_temp=pd.DataFrame()
    df_temp.columns = ['Tag','Lõigu sisu']
    df_temp['Pealkiri1'] = 'Poolte_nimed' 
    return df_temp

def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    df_temp['Kohtuasja_ID']='20429'
    return df_temp
tabel=kohtulahend(soup_from_txt)
tabel
#tabel.to_csv('Pohitabel.csv', mode='a', header=False, index=False)

In [23]:
#täiendavate andmete lisamine (2609-poolte nimed on tagi  C02RPAlineaAltA all)
# Sisse loetud!
soup_from_txt = BeautifulSoup(Path('DataET_puhas/2609kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt
def sisu_tabeliks(supp):
    abilist=[]
    tag_nimi=''
    Pealirjad_list=''
    Tekst_list=''
    Pealkirjad = [['C06Titre3'], ['C27Niveau6titre2'], ['C34Niveau7titre3'], ['Ctoc2'], ['C04Titre1'], ['CTOC4'], ['Ctoc1'], 
                  ['C06Titre4'], ['CTOC6'], ['C24Titre4'], ['C23Titre3'], ['C21Titrenumerote1'], ['C21Titre1'], 
                  ['C28Niveau6titre3'], ['C22Titrenumerote2'], ['C21Niveau5titre1'], ['C07Titre5'],['CTOC1'], 
                  ['C25Niveau5titre5'], ['Ctoc3'], ['C26Titre6'], ['C25Titrenumerote5'], ['CTOC2'], ['CTOC5'], 
                  ['C35Niveau7titre4'], ['C22Niveau5titre2'], ['C33Niveau7titre2'], ['CTOC3'], ['C24Titrenumerote4'], 
                  ['C22Titre2'], ['C30Niveau6titre5'], ['C23Niveau5titre3'], ['C26Titrenumerote6'], ['C05Titre2'], 
                  ['C29Niveau6titre4'], ['CTOC7'], ['C31Niveau6titre6'], ['C27Titrenumerote7'], ['Ctoc6'], ['C37Niveau7titre6'],
                  ['Cheading2'], ['Cheading1'], ['Cheading3'], ['C23Titrenumerote3'], ['C25Titre5'], ['Ctoc7'], ['C07Titre4'], 
                  ['Ctoc5'], ['C24Niveau5titre4'], ['C38Niveau7titre7'], ['C36Niveau7titre5'], ['C05Titre3'], ['Ctoc4'], 
                  ['C32Niveau7titre1'], ['C26Niveau6titre1']]
    Tekst = [['15Marge2avectiretlong'], ['C74Alineagrandespacement'], ['C12Tiretlong3'], ['C03Tiretlong'],
             ['Cnull'], ['C27Autre'], ['C39Autre'], ['C10Tiretlong1'], 
             ['C01PointAltN'], ['C50Alineaencadre'], ['C12Marge1avectiretlong'], ['C18Marge3avectiretlong'], ['C15Margeretrait0'],
             ['C20Centreitalique'], ['C38Marge1doubleretrait'], ['C80TableLibre'], ['C16Marge3'], ['C09Pointtypographiqueniv1'], 
             ['C28Pointnumerotemanuel'], ['C39autre'], ['C13Marge2'], ['C11Marge1avecretrait'], 
             ['C32Pointnumerotemanuel'], ['C11Tiretlong2'], ['C01PointnumeroteAltN'], ['C08Question'], ['C17Marge3avecretrait'], ['C09Marge0avecretrait'], ['C10Marge1'], ['C41Marge0doubleretrait'], 
             ['C29Marge0doubleretrait'], ['C14Marge2avecretrait'], ['C80Autre'],
             ['C40Pointnumerotemanuel']]
    for elem in supp.findAll('p'):
        if elem.has_attr('class'):
            #print(elem['class'])
            if elem['class'] in Pealkirjad:
                tag_nimi=elem['class'][0]
                Pealkirjad_list = elem.get_text().replace(u'\xa0', u' ')
                abilist.append([tag_nimi, Pealkirjad_list])
            elif elem['class'] in Tekst:
                Tekst_list = elem.get_text().replace(u'\xa0', u' ')
                tag_nimi=elem['class'][0]
                abilist.append([tag_nimi, Tekst_list])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag', 'Lõigu sisu']
    return df_temp
    

#pooled     
def poolte_nimed(supp):
    tag = 'C02RPAlineaAltA'
    abilist = []
    poolte_nimed = ''
    kogu_algus = supp.findAll('p', class_='C02RPAlineaAltA')
    poolte_nimed = []
    for elem in kogu_algus:
        if elem.findAll('b'):
            poolte_nimed=elem.get_text()
            abilist.append([tag, poolte_nimed])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag','Lõigu sisu']
    df_temp['Pealkiri1'] = 'Poolte_nimed' 
    return df_temp

def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    df_temp['Kohtuasja_ID']='2609'
    return df_temp#.iloc[1]['Lõigu sisu']
tabel=kohtulahend(soup_from_txt)
tabel
#tabel.to_csv('Pohitabel.csv', mode='a', header=False, index=False)

In [42]:
# 2383 Märksõnad on c19Centre all, sisseloetud õigesti.
soup_from_txt = BeautifulSoup(Path('DataET_puhas/2383kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt

def viited_marksonale(supp):
    viited = supp.findAll('p', class_='C19Centre')
    #print(viited)
    viited1 = viited[2].get_text().replace(u'\xa0', u' ')
    df_temp=pd.DataFrame(columns=['Lõigu sisu','Pealkiri1'], index=range(1))
    df_temp['Lõigu sisu'] = viited1
    df_temp['Pealkiri1'] = 'Viited'
    df_temp['Tag'] = 'C71Indicateur'
    return df_temp

def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    df_temp['Kohtuasja_ID']='2383'
    return df_temp
tabel=kohtulahend(soup_from_txt)
tabel
#tabel.to_csv('Pohitabel.csv', mode='a', header=False, index=False)

In [93]:
#täiendavate andmete lisamine (4552, 5024, 6008, 6106, 6694, 7602, 7693 -poolte nimed on C04Titre1 all)
# Sisse loetud!
soup_from_txt = BeautifulSoup(Path('DataET_puhas/7693kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt
def sisu_tabeliks(supp):
    abilist=[]
    tag_nimi=''
    Pealirjad_list=''
    Tekst_list=''
    Pealkirjad = [['C06Titre3'], ['C27Niveau6titre2'], ['C34Niveau7titre3'], ['Ctoc2'], ['C04Titre1'], ['CTOC4'], ['Ctoc1'], 
                  ['C06Titre4'], ['CTOC6'], ['C24Titre4'], ['C23Titre3'], ['C21Titrenumerote1'], ['C21Titre1'], 
                  ['C28Niveau6titre3'], ['C22Titrenumerote2'], ['C21Niveau5titre1'], ['C07Titre5'],['CTOC1'], 
                  ['C25Niveau5titre5'], ['Ctoc3'], ['C26Titre6'], ['C25Titrenumerote5'], ['CTOC2'], ['CTOC5'], 
                  ['C35Niveau7titre4'], ['C22Niveau5titre2'], ['C33Niveau7titre2'], ['CTOC3'], ['C24Titrenumerote4'], 
                  ['C22Titre2'], ['C30Niveau6titre5'], ['C23Niveau5titre3'], ['C26Titrenumerote6'], ['C05Titre2'], 
                  ['C29Niveau6titre4'], ['CTOC7'], ['C31Niveau6titre6'], ['C27Titrenumerote7'], ['Ctoc6'], ['C37Niveau7titre6'],
                  ['Cheading2'], ['Cheading1'], ['Cheading3'], ['C23Titrenumerote3'], ['C25Titre5'], ['Ctoc7'], ['C07Titre4'], 
                  ['Ctoc5'], ['C24Niveau5titre4'], ['C38Niveau7titre7'], ['C36Niveau7titre5'], ['C05Titre3'], ['Ctoc4'], 
                  ['C32Niveau7titre1'], ['C26Niveau6titre1']]
    Tekst = [['15Marge2avectiretlong'], ['C74Alineagrandespacement'], ['C12Tiretlong3'], ['C03Tiretlong'],
             ['Cnull'], ['C27Autre'], ['C39Autre'], ['C10Tiretlong1'], 
             ['C01PointAltN'], ['C50Alineaencadre'], ['C12Marge1avectiretlong'], ['C18Marge3avectiretlong'], ['C15Margeretrait0'],
             ['C20Centreitalique'], ['C38Marge1doubleretrait'], ['C80TableLibre'], ['C16Marge3'], ['C09Pointtypographiqueniv1'], 
             ['C28Pointnumerotemanuel'], ['C39autre'], ['C13Marge2'], ['C11Marge1avecretrait'], 
             ['C32Pointnumerotemanuel'], ['C11Tiretlong2'], ['C01PointnumeroteAltN'], ['C08Question'], ['C17Marge3avecretrait'], ['C09Marge0avecretrait'], ['C10Marge1'], ['C41Marge0doubleretrait'], 
             ['C29Marge0doubleretrait'], ['C14Marge2avecretrait'], ['C80Autre'],
             ['C40Pointnumerotemanuel']]
    for elem in supp.findAll('p'):
        if elem.has_attr('class'):
            #print(elem['class'])
            if elem['class'] in Pealkirjad:
                tag_nimi=elem['class'][0]
                Pealkirjad_list = elem.get_text().replace(u'\xa0', u' ')
                abilist.append([tag_nimi, Pealkirjad_list])
            elif elem['class'] in Tekst:
                Tekst_list = elem.get_text().replace(u'\xa0', u' ')
                tag_nimi=elem['class'][0]
                abilist.append([tag_nimi, Tekst_list])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag', 'Lõigu sisu']
    return df_temp
    

#pooled     
def poolte_nimed(supp):
    tag = 'C04Titre1'
    abilist = []
    poolte_nimed = ''
    kogu_algus = supp.findAll('p', class_='C04Titre1')
    poolte_nimed = []
    for elem in kogu_algus:
        if elem.findAll('b'):
            poolte_nimed=elem.get_text()
            abilist.append([tag, poolte_nimed])
    df_temp=pd.DataFrame(abilist[:2])
    df_temp.columns = ['Tag','Lõigu sisu']
    df_temp['Pealkiri1'] = 'Poolte_nimed' 
    return df_temp

def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    df_temp['Kohtuasja_ID']='7693'
    return df_temp#.iloc[1]['Lõigu sisu']
tabel=kohtulahend(soup_from_txt)
tabel
#tabel.to_csv('Pohitabel.csv', mode='a', header=False, index=False)

In [99]:
#täiendavate andmete lisamine (6428, 6485  -poolte nimed on C30Dispositifalinea all)
# Sisse loetud!
soup_from_txt = BeautifulSoup(Path('DataET_puhas/6485kohtulahend.txt').read_text(encoding='utf-8'), 'html.parser')
soup_from_txt
def sisu_tabeliks(supp):
    abilist=[]
    tag_nimi=''
    Pealirjad_list=''
    Tekst_list=''
    Pealkirjad = [['C06Titre3'], ['C27Niveau6titre2'], ['C34Niveau7titre3'], ['Ctoc2'], ['C04Titre1'], ['CTOC4'], ['Ctoc1'], 
                  ['C06Titre4'], ['CTOC6'], ['C24Titre4'], ['C23Titre3'], ['C21Titrenumerote1'], ['C21Titre1'], 
                  ['C28Niveau6titre3'], ['C22Titrenumerote2'], ['C21Niveau5titre1'], ['C07Titre5'],['CTOC1'], 
                  ['C25Niveau5titre5'], ['Ctoc3'], ['C26Titre6'], ['C25Titrenumerote5'], ['CTOC2'], ['CTOC5'], 
                  ['C35Niveau7titre4'], ['C22Niveau5titre2'], ['C33Niveau7titre2'], ['CTOC3'], ['C24Titrenumerote4'], 
                  ['C22Titre2'], ['C30Niveau6titre5'], ['C23Niveau5titre3'], ['C26Titrenumerote6'], ['C05Titre2'], 
                  ['C29Niveau6titre4'], ['CTOC7'], ['C31Niveau6titre6'], ['C27Titrenumerote7'], ['Ctoc6'], ['C37Niveau7titre6'],
                  ['Cheading2'], ['Cheading1'], ['Cheading3'], ['C23Titrenumerote3'], ['C25Titre5'], ['Ctoc7'], ['C07Titre4'], 
                  ['Ctoc5'], ['C24Niveau5titre4'], ['C38Niveau7titre7'], ['C36Niveau7titre5'], ['C05Titre3'], ['Ctoc4'], 
                  ['C32Niveau7titre1'], ['C26Niveau6titre1']]
    Tekst = [['15Marge2avectiretlong'], ['C74Alineagrandespacement'], ['C12Tiretlong3'], ['C03Tiretlong'],
             ['Cnull'], ['C27Autre'], ['C39Autre'], ['C10Tiretlong1'], 
             ['C01PointAltN'], ['C50Alineaencadre'], ['C12Marge1avectiretlong'], ['C18Marge3avectiretlong'], ['C15Margeretrait0'],
             ['C20Centreitalique'], ['C38Marge1doubleretrait'], ['C80TableLibre'], ['C16Marge3'], ['C09Pointtypographiqueniv1'], 
             ['C28Pointnumerotemanuel'], ['C39autre'], ['C13Marge2'], ['C11Marge1avecretrait'], 
             ['C32Pointnumerotemanuel'], ['C11Tiretlong2'], ['C01PointnumeroteAltN'], ['C08Question'], ['C17Marge3avecretrait'], ['C09Marge0avecretrait'], ['C10Marge1'], ['C41Marge0doubleretrait'], 
             ['C29Marge0doubleretrait'], ['C14Marge2avecretrait'], ['C80Autre'],
             ['C40Pointnumerotemanuel']]
    for elem in supp.findAll('p'):
        if elem.has_attr('class'):
            #print(elem['class'])
            if elem['class'] in Pealkirjad:
                tag_nimi=elem['class'][0]
                Pealkirjad_list = elem.get_text().replace(u'\xa0', u' ')
                abilist.append([tag_nimi, Pealkirjad_list])
            elif elem['class'] in Tekst:
                Tekst_list = elem.get_text().replace(u'\xa0', u' ')
                tag_nimi=elem['class'][0]
                abilist.append([tag_nimi, Tekst_list])
    df_temp=pd.DataFrame(abilist)
    df_temp.columns = ['Tag', 'Lõigu sisu']
    return df_temp
    

#pooled     
def poolte_nimed(supp):
    tag = 'C30Dispositifalinea'
    abilist = []
    poolte_nimed = ''
    kogu_algus = supp.findAll('p', class_='C30Dispositifalinea')
    poolte_nimed = []
    for elem in kogu_algus:
        if elem.findAll('b'):
            poolte_nimed=elem.get_text()
            abilist.append([tag, poolte_nimed])
    df_temp=pd.DataFrame(abilist[:2])
    df_temp.columns = ['Tag','Lõigu sisu']
    df_temp['Pealkiri1'] = 'Poolte_nimed' 
    return df_temp

def kohtulahend(supp):
    df_temp = pd.DataFrame(columns=['Tag','Lõigu sisu','Pealkiri1', 'Kohtuasja_nr', "Kohtuasja_ID"])
    df_temp = df_temp.append(koda(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kuupaev(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(viited_marksonale(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(poolte_nimed(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(kohtumenetluse_keel(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(sisu_tabeliks(supp), ignore_index=True, sort=True)
    df_temp = df_temp.append(resolutsioon(supp), ignore_index=True, sort=True)
    df_temp['Kohtuasja_nr']=kohtuasja_nr(supp)
    df_temp['Kohtuasja_ID']='6485'
    return df_temp#.iloc[1]['Lõigu sisu']
tabel=kohtulahend(soup_from_txt)
tabel
#tabel.to_csv('Pohitabel.csv', mode='a', header=False, index=False)